In [22]:
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import SparkSession

   # Создаем SparkSession
spark = SparkSession.builder.appName("SmokeDetection").getOrCreate()

   # Загружаем данные с помощью Pandas
pandas_df = pd.read_csv("smoke_detection_iot.csv")
  # Загружаем данные с помощ spark
df = spark.read.csv("smoke_detection_iot.csv", header = "True", inferSchema=True) 

In [25]:
# Выбираем нужные колонки (предположим, нас интересует предсказание температуры)
data = df.select(
    col("Temperature[C]").alias("label"),
    "Humidity[%]",
    "TVOC[ppb]",
    "eCO2[ppm]",
    "Raw H2",
    "Raw Ethanol",
)

In [26]:
# 1. Заполняем пропуски (если есть) средним значением
for col_name in data.columns[1:]:
    mean_value = data.agg({col_name: "mean"}).collect()[0][0]
    data = data.fillna(mean_value, subset=[col_name])

# 2. Собираем фичи в вектор
assembler = VectorAssembler(inputCols=data.columns[1:], outputCol="features")
data = assembler.transform(data)

# 3. Масштабируем фичи
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)
scalerModel = scaler.fit(data)
data = scalerModel.transform(data)


In [27]:
train_data, test_data = data.randomSplit([0.7, 0.3])



In [28]:
lr = LinearRegression(featuresCol="scaledFeatures", labelCol="label")
lr_model = lr.fit(train_data)


2024-06-16 04:03:24,338 WARN util.Instrumentation: [dc92c439] regParam is zero, which might cause numerical instability and overfitting.
2024-06-16 04:03:26,108 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2024-06-16 04:03:26,109 WARN netlib.BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
2024-06-16 04:03:26,916 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
2024-06-16 04:03:26,916 WARN netlib.LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


In [29]:
predictions = lr_model.transform(test_data)


In [30]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) = %g" % rmse)


Root Mean Squared Error (RMSE) = 13.8546
